In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization library  
import statistics as stats # https://docs.python.org/3/library/statistics.html#statistics.fmean
#import scipy.stats as spstats
import matplotlib.pyplot as plt

### Validation & Normalization methods ###
from sklearn.model_selection import cross_validate, StratifiedKFold, RepeatedStratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

### ML models ###
from sklearn.linear_model import LogisticRegression, SGDClassifier # C1 loss: log_loss => LogisticRegression with SGD
from sklearn.linear_model import Perceptron # C2
from sklearn.svm import NuSVC # C3
from sklearn.svm import LinearSVC # C4
from sklearn.tree import DecisionTreeClassifier # C5
from sklearn.ensemble import RandomForestClassifier # C6
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier # C7

### Metrics ###
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, make_scorer
from imblearn.metrics import geometric_mean_score # https://imbalanced-learn.org/stable/references/generated/imblearn.metrics.geometric_mean_score.html
import time
import timeit # https://stackoverflow.com/questions/17579357/time-time-vs-timeit-timeit

### Pipeline ###
from sklearn.pipeline import make_pipeline , Pipeline # https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

### Analysis ###
from sklearn.feature_selection import SelectKBest, f_classif, chi2, RFE, RFECV , mutual_info_classif

### Custom Modules ###
from functions.data_types import optimize_dtypes
from functions.dataframe_actions import df_info, df_clean, show_value_counts, fill_missing_values
from functions.ml_training import train_classifiers, train_classifiers_tuned, train_evaluate_single
from functions.dimensionality_reduction import apply_pca, plot_variance

### Other configurations ###
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
# #import warnings library
# import warnings
# # ignore all warnings
# warnings.filterwarnings('ignore')

from sklearn import set_config
set_config(working_memory=1024*20) 

from sklearn.cluster import AgglomerativeClustering, FeatureAgglomeration
from sklearn.decomposition import PCA, SparsePCA, KernelPCA, TruncatedSVD

/home/lazaros/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# to read .csv files from another directory
data_location = "../Data/" # "/<path>"

df_train = pd.read_csv(data_location + "train_oversampled_smote.csv")
df_train = optimize_dtypes(df_train)
# df_train.head()

df_test = pd.read_csv(data_location + "test_filled_mapped.csv")
df_test = optimize_dtypes(df_test)


# Separate target variable from feature variables
X_train = df_train.drop('HadHeartAttack', axis=1, inplace=False)  # Features
y_train = df_train['HadHeartAttack']

# Separate target variable from feature variables
X_test = df_test.drop('HadHeartAttack', axis=1, inplace=False)  # Features
y_test = df_test['HadHeartAttack']   

In [3]:
original_dtypes = X_train.dtypes

X_train_scaled = (X_train - X_train.min(axis=0)) / (X_train.max(axis=0)-X_train.min(axis=0))              # min max scale
# X_train_scaled = (X_train - X_train.mean())/X_train.std() # If we use StandardScaler, the feature names will be lost, so we do it mannually.

# x_scaled.hist(figsize=(16, 20), bins=30, edgecolor="black") # plot to show features after scaling
# plt.subplots_adjust()

# Cast back to original data types
for column, dtype in original_dtypes.items():
    X_train_scaled[column] = X_train_scaled[column].astype(dtype)


original_dtypes = X_test.dtypes

# Apply Min-Max scaling based on training set statistics
X_test_scaled = (X_test - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))

# Cast back to original data types
for column, dtype in original_dtypes.items():
    X_test_scaled[column] = X_test_scaled[column].astype(dtype)

In [4]:
# # Define multiple metrics
# scoring = {'Balanced Accuracy': make_scorer(balanced_accuracy_score),
#            'F1-score': make_scorer(f1_score, average='weighted'),
#            'G-Mean score': make_scorer(geometric_mean_score, average='weighted')
#           }

## PCA

In [5]:
pca = PCA(n_components=18, svd_solver='full', random_state=13).fit(X_train_scaled)
X_pca_train = pca.transform(X_train_scaled)
X_pca_test = pca.transform(X_test_scaled)

In [6]:
rf = RandomForestClassifier(criterion='entropy', n_jobs=-1, class_weight='balanced', random_state=13)
metrics_rf = train_evaluate_single(rf, X_pca_train, y_train, X_pca_test, y_test, classifier_name="RF")

# Store metrics in a DataFrame
metrics_df_rf = pd.DataFrame([metrics_rf])
metrics_df_rf

,classifier,accuracy,balanced_accuracy,precision,recall,f1_score,roc_auc,fit_time
0,RF,0.885799,0.678785,0.23421,0.445241,0.306953,0.829059,132.116985


In [7]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=-1)
metrics_knn = train_evaluate_single(knn, X_pca_train, y_train, X_pca_test, y_test, classifier_name="kNN")

# Store metrics in a DataFrame
metrics_df_knn = pd.DataFrame([metrics_knn])
metrics_df_knn

# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

,classifier,accuracy,balanced_accuracy,precision,recall,f1_score,roc_auc,fit_time
0,kNN,0.812349,0.688877,0.161507,0.549582,0.249649,0.761197,0.043076


In [8]:
qda = QuadraticDiscriminantAnalysis(priors=None, reg_param=0.7, store_covariance=False, tol=0.0001)
metrics_qda = train_evaluate_single(qda, X_pca_train, y_train, X_pca_test, y_test, classifier_name="QDA")

# Store metrics in a DataFrame
metrics_df_qda = pd.DataFrame([metrics_qda])
metrics_df_qda

# https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html#sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis

,classifier,accuracy,balanced_accuracy,precision,recall,f1_score,roc_auc,fit_time
0,QDA,0.772219,0.761638,0.166247,0.749701,0.272146,0.840593,0.869553


In [9]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate_init=0.001, max_iter=1000, shuffle=True, random_state=13, tol=0.0001, verbose=False, warm_start=False, early_stopping=True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=20)
metrics_mlp = train_evaluate_single(mlp, X_pca_train, y_train, X_pca_test, y_test, classifier_name="MLP")

# Store metrics in a DataFrame
metrics_df_mlp = pd.DataFrame([metrics_mlp])
metrics_df_mlp

# https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html#sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis

,classifier,accuracy,balanced_accuracy,precision,recall,f1_score,roc_auc,fit_time
0,MLP,0.825989,0.724085,0.185608,0.60912,0.284518,0.820218,109.413078


In [10]:
pca_smote_results = pd.concat([metrics_df_rf, metrics_df_knn, metrics_df_qda, metrics_df_mlp])
pca_smote_results.to_csv('../Data/results_pca_smote.csv', index=False)
pca_smote_results

,classifier,accuracy,balanced_accuracy,precision,recall,f1_score,roc_auc,fit_time
0,RF,0.885799,0.678785,0.234210,0.445241,0.306953,0.829059,132.116985
0,kNN,0.812349,0.688877,0.161507,0.549582,0.249649,0.761197,0.043076
0,QDA,0.772219,0.761638,0.166247,0.749701,0.272146,0.840593,0.869553
0,MLP,0.825989,0.724085,0.185608,0.609120,0.284518,0.820218,109.413078


## Sparse PCA

In [12]:
pca = SparsePCA(n_components=18, alpha=1, ridge_alpha=0.01, max_iter=1, tol=1e-08, method='lars', n_jobs=-1, U_init=None, V_init=None, verbose=False, random_state=13).fit(X_train_scaled)
X_pca_train = pca.transform(X_train_scaled)
X_pca_test = pca.transform(X_test_scaled)

In [ ]:
rf = RandomForestClassifier(criterion='entropy', n_jobs=-1, class_weight='balanced', random_state=13)
metrics_rf = train_evaluate_single(rf, X_pca_train, y_train, X_pca_test, y_test, classifier_name="RF")

# Store metrics in a DataFrame
metrics_df_rf = pd.DataFrame([metrics_rf])
metrics_df_rf

In [13]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=-1)
metrics_knn = train_evaluate_single(knn, X_pca_train, y_train, X_pca_test, y_test, classifier_name="kNN")

# Store metrics in a DataFrame
metrics_df_knn = pd.DataFrame([metrics_knn])
metrics_df_knn

# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

,classifier,accuracy,balanced_accuracy,precision,recall,f1_score,roc_auc,fit_time
0,kNN,0.812349,0.688503,0.161349,0.548785,0.249378,0.761185,0.042673


In [14]:
qda = QuadraticDiscriminantAnalysis(priors=None, reg_param=0.7, store_covariance=False, tol=0.0001)
metrics_qda = train_evaluate_single(qda, X_pca_train, y_train, X_pca_test, y_test, classifier_name="QDA")

# Store metrics in a DataFrame
metrics_df_qda = pd.DataFrame([metrics_qda])
metrics_df_qda

# https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html#sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis

,classifier,accuracy,balanced_accuracy,precision,recall,f1_score,roc_auc,fit_time
0,QDA,0.772072,0.76156,0.166152,0.749701,0.272018,0.840587,0.895165


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate_init=0.001, max_iter=1000, shuffle=True, random_state=13, tol=0.0001, verbose=False, warm_start=False, early_stopping=True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=20)
metrics_mlp = train_evaluate_single(mlp, X_pca_train, y_train, X_pca_test, y_test, classifier_name="MLP")

# Store metrics in a DataFrame
metrics_df_mlp = pd.DataFrame([metrics_mlp])
metrics_df_mlp

# https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html#sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis

In [ ]:
pca_smote_results = pd.concat([metrics_df_rf, metrics_df_knn, metrics_df_qda, metrics_df_mlp])
pca_smote_results.to_csv('../Data/results_sparse_pca_smote.csv', index=False)
pca_smote_results

## Truncated SVD

In [15]:
pca = TruncatedSVD(n_components=18, algorithm='arpack', random_state=13, tol=0.0).fit(X_train_scaled)
X_pca_train = pca.transform(X_train_scaled)
X_pca_test = pca.transform(X_test_scaled)

ValueError: matrix type must be 'f', 'd', 'F', or 'D'

## Kernel PCA

In [16]:
pca = KernelPCA(n_components=18, kernel='linear', gamma=None, degree=3, coef0=1, kernel_params=None, alpha=1.0, fit_inverse_transform=False, eigen_solver='auto', tol=0, max_iter=None, iterated_power='auto', remove_zero_eig=False, random_state=13, copy_X=True, n_jobs=-1).fit(X_train_scaled)
X_pca_train = pca.transform(X_train_scaled)
X_pca_test = pca.transform(X_test_scaled)

MemoryError: Unable to allocate 3.24 TiB for an array with shape (667134, 667134) and data type float64